In [20]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

!pip install gspread==5.3.2
import gspread
from google.oauth2.service_account import Credentials

In [2]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    '/content/project-ds-347713-0717d5778e6b.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)


In [3]:
file= gc.open('Raw Parkiran PT. SUMO UTOMO')
name_sheet = ['Jan-2021','Feb-2021','Mar-2021','Apr-2021','Mei-2021','Jun-2021','Jul-2021','Agu-2021','Sep-2021','Okt-2021','Nov-2021','Des-2021']

data = pd.DataFrame(columns=['Gate','Vehicle','Entry','Exit'])

for i in name_sheet:
  sheet = file.worksheet(i)
  df = pd.DataFrame(sheet.get_all_records())
  data = data.append(df)

data['Entry'] = pd.to_datetime(data['Entry'])
data['Exit'] = pd.to_datetime(data['Exit'])
data['Month'] = data['Entry'].dt.month_name()

### 1. Persentase transaksi error tiap bulan dan keseluruhan

In [4]:
data['Validasi'] = data['Entry'] < data['Exit']
hasil_1 = data.groupby(['Month' , 'Validasi']).agg({'Entry':'count'}).unstack().reset_index()
hasil_1.columns = ['Month', 'FALSE', 'TRUE']
hasil_1['PERSENTASE'] = (hasil_1['FALSE']/(hasil_1['FALSE']+hasil_1['TRUE']))*100
hasil_1

,Month,FALSE,TRUE,PERSENTASE
0,April,53,47,53.0
1,August,54,46,54.0
2,December,57,43,57.0
3,February,46,54,46.0
4,January,48,52,48.0
5,July,55,45,55.0
6,June,54,46,54.0
7,March,46,54,46.0
8,May,41,59,41.0
9,November,53,47,53.0


###2. Pendapatan Transaksi Tiap Bulan

In [5]:
df = data[data['Validasi'] == True].reset_index()
df.head(2)

,index,Gate,Vehicle,Entry,Exit,Month,Validasi
0,0,B2,Motorcycle,2021-01-06 22:23:57,2021-01-19 23:45:02,January,True
1,2,A1,Car,2021-01-06 04:47:47,2021-01-24 00:58:50,January,True


In [6]:
def get_hours(end,start):
  ga = (end-start)
  minutes = ga.total_seconds()/60
  hours = minutes/60
  days = hours/24

  return math.ceil(hours)

for index, row in df.iterrows():
  df.loc[index,'Duration'] = get_hours(row['Exit'], row['Entry'])

df.loc[df['Vehicle'] == 'Car', 'Price'] = df['Duration']*3000
df.loc[df['Vehicle'] == 'Motorcycle', 'Price'] = df['Duration']*2000

In [7]:
hasil_2 = df.groupby(['Month']).agg({'Price':'sum'}).reset_index()
hasil_2

,Month,Price
0,April,30853000.0
1,August,30605000.0
2,December,30419000.0
3,February,29484000.0
4,January,29871000.0
5,July,19737000.0
6,June,23418000.0
7,March,31814000.0
8,May,41624000.0
9,November,32319000.0


###3. Berapa Pendapatan Tiap Bulan Berdasarkan jenis kendaraan

In [11]:
df.groupby('Month')['Price'].sum()

Month
April        30853000.0
August       30605000.0
December     30419000.0
February     29484000.0
January      29871000.0
July         19737000.0
June         23418000.0
March        31814000.0
May          41624000.0
November     32319000.0
October      32611000.0
September    24206000.0
Name: Price, dtype: float64

In [14]:
df.groupby(['Month', 'Vehicle']).agg({'Price':'sum'}).reset_index()

,Month,Vehicle,Price
0,April,Car,18351000.0
1,April,Motorcycle,12502000.0
2,August,Car,14367000.0
3,August,Motorcycle,16238000.0
4,December,Car,23067000.0
5,December,Motorcycle,7352000.0
6,February,Car,13974000.0
7,February,Motorcycle,15510000.0
8,January,Car,13065000.0
9,January,Motorcycle,16806000.0
